## Read Text Data From File
<ul>
<li> remove label 'handle', 'tweet'</li>
    <li> encoding = 'utf-8'</li>
    <li> labelList:  Clinton: 1 ; Trump: 0</li>
    <li> print size</li>
</ul>

In [23]:
## read text data from file

import csv
import string

labelList = []
tweetList = []
tweetList_test = []

with open('train.csv', encoding='utf-8') as trainfile:
    reader = csv.reader(trainfile)
    for row in reader:
        if row[0] == 'HillaryClinton':
            labelList.append(1)
        elif row[0] == 'realDonaldTrump':
            labelList.append(0)
            
        if row[1] != 'tweet':
            tweetList.append(row[1])
            
with open('test.csv', encoding='utf-8') as testfile:
    reader = csv.reader(testfile)
    for row in reader: 
        if row[1] != 'tweet':
            tweetList_test.append(row[1])

N = len(tweetList)
print(len(tweetList))
print(len(labelList))

N_test = len(tweetList_test)
print(len(tweetList_test))

4743
4743
1701


## Create Corpus
<ul>
    <li>unique word in corpus</li>
</ul>

In [25]:
## corpus
import string

def refineWordsList(tweetList):
    ret = []
    for tweet in tweetList:
        step1 = tweet.split('http',1)[0]
        step2 = step1.lower()
        step3 = step2.translate(str.maketrans('.',' '))
        step4 = step3.translate(str.maketrans('','',string.punctuation))
        ret.append(step4)
    return ret

tweetList = refineWordsList(tweetList)
tweetList_test = refineWordsList(tweetList_test)

corpus = set()

for tweet in tweetList:
    words = tweet.split();
    corpus.update(words)

corpus = list(corpus)
M = len(corpus)
print(M)

8787


## Label Encoding

<ul>
    <li>hash table encoder each word to unique number</li>
    <li>encode entire tweet list</li>
    <li>decode method</li>
</ul>

In [27]:
## Label Encoding

hashEncoder = {}
i = 0
for word in corpus:
    hashEncoder[word] = i
    i += 1

# train
tweetList_encoded = []
for tweet in tweetList:
    tweet_encoded = []
    for word in tweet.split():
        word_encoded = hashEncoder[word]
        tweet_encoded.append(word_encoded)
    tweetList_encoded.append(tweet_encoded)
    
# test
tweetList_test_encoded = []
for tweet in tweetList_test:
    tweet_encoded = []
    for word in tweet.split():
        if hashEncoder.get(word) != None :
            word_encoded = hashEncoder[word]
            tweet_encoded.append(word_encoded)
    tweetList_test_encoded.append(tweet_encoded)

def decodeTweet(tweet_encoded):
    decodeList = []
    for word_encoded in tweetList_encoded[5]:
        decodeList.append(list(hashEncoder.keys())[list(hashEncoder.values()).index(word_encoded)])
    return decodeList
    
print(len(tweetList_encoded))
print(len(tweetList_test_encoded))
print()
print(tweetList[5])
print(tweetList_encoded[5])
print(decodeTweet(tweetList_encoded[5]))
print()
print(tweetList_test[5])
print(tweetList_test_encoded[5])

4743
1701

when donald trump goes low   register to vote 
[362, 6425, 5398, 1522, 85, 4650, 670, 3828]
['when', 'donald', 'trump', 'goes', 'low', 'register', 'to', 'vote']

its nationalvoterregistrationday  celebrate by registering to vote → 
[309, 166, 3577, 2913, 4089, 670, 3828, 3422]


## import

In [10]:
import tensorflow as tf
import numpy as np
import time
import math
from tensorflow.contrib import rnn

## LSTM

## FeedForwardNN

In [ ]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])